#Flask App

In [1]:
%%writefile solarApp.py
from flask import Flask
from flask import request
app = Flask(__name__)
from flask import render_template
from solarApp_helper_functions import return_power

@app.route('/', methods=['POST','GET'])
def submission_page():
    return render_template('template.html')

@app.route('/about')
def about_page():
    return render_template('about.html')

@app.route('/output', methods=['POST','GET'] )
def output():
    
    #form data
    panel_type_data = request.form['panel_type']
    number_panels_data = int(request.form['number_panels'])
    month_data = int(request.form['month'])
    day_data = int(request.form['day'])
    hour_data = int(request.form['hour'])
    
    #pred power calculation and getting satellite image into foo.png
    predicted_power = return_power(month_data, day_data, hour_data) / 66.0 * number_panels_data
    
    return render_template('output.html', panel_type_data=panel_type_data, 
                           number_panels_data=number_panels_data, month_data=month_data, 
                           day_data=day_data, hour_data=hour_data, 
                           predicted_power=int(predicted_power))

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=80, debug=False)

Overwriting solarApp.py


In [ ]:
#!python solarApp.py
!sudo -E python solarApp.py

 * Running on http://0.0.0.0:80/ (Press CTRL+C to quit)
 * Restarting with stat
128.177.113.106 - - [19/Aug/2015 03:31:26] "GET / HTTP/1.1" 200 -
128.177.113.106 - - [19/Aug/2015 03:31:26] "GET /static/bootstrap-3.3.5-dist/css/bootstrap.min.css HTTP/1.1" 304 -
128.177.113.106 - - [19/Aug/2015 03:31:26] "GET /static/bootstrap-3.3.5-dist/css/bootstrap-theme.min.css HTTP/1.1" 304 -
128.177.113.106 - - [19/Aug/2015 03:31:26] "GET /static/bootstrap-3.3.5-dist/js/bootstrap.min.js HTTP/1.1" 304 -
128.177.113.106 - - [19/Aug/2015 03:31:26] "GET /dist/js/bootstrap.min.js HTTP/1.1" 404 -
128.177.113.106 - - [19/Aug/2015 03:31:26] "GET /assets/js/ie10-viewport-bug-workaround.js HTTP/1.1" 404 -
/home/ubuntu/anaconda/lib/python2.7/site-packages/scipy/ndimage/interpolation.py:549: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)
128.17

#Build a model

In [ ]:
from __future__ import division
from scipy.ndimage import zoom
from random import randint
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime, timedelta, time
from data_helper_functions_webapp import *
from time import sleep

month_data, day_data, hour_data = (4,5,19)
'''Input: datetime
Output: power
Info: also makes satellite image'''

######### Satellite image ###############
# get sat image first, so it may be redered by computation is done
desired_channel = 'BAND_01'
desired_date = datetime(2014, month_data, day_data)
desired_timedelta = timedelta(hours = hour_data)
desired_datetime = desired_date + desired_timedelta
satellite_filefolder = '../../data/satellite/colorado/summer6months/data/'
sensor_filefolder = '../../data/sensor_data/colorado6months/'
pvoutput_filefolder = '../../data/pvoutput/pvoutput6months/'

#satellite data
satellite_filename = find_filename(desired_datetime, desired_channel, satellite_filefolder)
lons, lats, data = return_satellite_data(satellite_filename, satellite_filefolder)

plt.figure(figsize=(8, 8))
imgplot = plt.imshow(data)
imgplot.set_interpolation('none')
plt.savefig('static/images/foo.png', bbox_inches='tight') # save sat image to foo.png

############## Model for satellite to sensor ############################

X = [] #sat data
Y = [] #sensor data

desired_date = (desired_datetime - timedelta(hours=6)).date() #make sure correct date
desired_date = datetime.combine(desired_date, time.min) #get into datetime format

desired_channel = 'BAND_01' #problems with an inner for loop (doesn't look good, but works)
satellite_filename = find_filename(desired_datetime, desired_channel, satellite_filefolder)
lons, lats, data1 = return_satellite_data(satellite_filename, satellite_filefolder)

desired_channel = 'BAND_02'
satellite_filename = find_filename(desired_datetime, desired_channel, satellite_filefolder)
lons, lats, data2 = return_satellite_data(satellite_filename, satellite_filefolder)

desired_channel = 'BAND_03'
satellite_filename = find_filename(desired_datetime, desired_channel, satellite_filefolder)
lons, lats, data3 = return_satellite_data(satellite_filename, satellite_filefolder)               

desired_channel = 'BAND_04'
satellite_filename = find_filename(desired_datetime, desired_channel, satellite_filefolder)
lons, lats, data4 = return_satellite_data(satellite_filename, satellite_filefolder)

desired_channel = 'BAND_06'
satellite_filename = find_filename(desired_datetime, desired_channel, satellite_filefolder)
lons, lats, data5 = return_satellite_data(satellite_filename, satellite_filefolder)

sensor_filename = find_file_from_date(desired_date, sensor_filefolder)
df_sensor = return_sensor_data(sensor_filename, sensor_filefolder).ix[:,-15:-1]
df_sensor[df_sensor.index == desired_datetime]

Y.append(df_sensor[df_sensor.index == desired_datetime].values[0])
X.append(np.hstack( ( np.ravel(data1) , np.ravel(data2), np.ravel(data3) , np.ravel(data4), np.ravel(data5) ) ) )

X,Y = (np.array(X),np.array(Y))

####################### Make sat data useful ####################
X_ratio_1_2 = []
for i in xrange(X.shape[0]): #a little awkward since X is only one row, but no need to change
    CH1 = zoom(X[:,0:1972][i].reshape((29,68)),zoom=(0.48, 0.53), order=5)
    CH2 = X[:,1972:2476][i].reshape((14,36))
    X_ratio_1_2.append(25000* (CH2)  / (CH1 + CH2+1.0) )
X_ratio_1_2 = np.array(X_ratio_1_2)

X_ratio_1_6 = []
for i in xrange(X.shape[0]):
    CH1 = zoom(X[:,0:1972][i].reshape((29,68)),zoom=(0.48, 0.53), order=5)
    CH6 = X[:,3484:3988][i].reshape((14,36))
    X_ratio_1_6.append(25000* CH6 / (CH1 + CH6 + 0.1) )
X_ratio_1_6 = np.array(X_ratio_1_6)

X_ratio_2_6 = []
for i in xrange(X.shape[0]):
    CH2 = X[:,1972:2476][i].reshape((14,36))
    CH6 = X[:,3484:3988][i].reshape((14,36))
    X_ratio_2_6.append(25000* CH6 / (CH2 + CH6 + 0.1) )
X_ratio_2_6 = np.array(X_ratio_2_6)

######## change X into histogram #############
X_hist = []
bins = 25
for i in xrange(X.shape[0]):
    myval1 = pd.DataFrame(np.ravel(X_ratio_1_2[i])).fillna(np.mean).values.flatten();
    myval2 = pd.DataFrame(np.ravel(X_ratio_1_6[i])).fillna(np.mean).values.flatten();
    myval3 = pd.DataFrame(np.ravel(X_ratio_2_6[i])).fillna(np.mean).values.flatten();

    hist1, _ = np.histogram(X[:,0:1972][i], density=True, bins=bins, range=(0,25000))
    hist2, _ = np.histogram(X[:,1972:2476][i], density=True, bins=bins, range=(0,25000))
    hist3, _ = np.histogram(X[:,2476:2980][i], density=True, bins=bins, range=(0,25000))
    hist4, _ = np.histogram(X[:,2980:3484][i], density=True, bins=bins, range=(0,25000))
    hist5, _ = np.histogram(X[:,3484:3988][i], density=True, bins=bins, range=(0,25000))
    hist6, _ = np.histogram( myval1 , density=True, bins=bins, range=(0,25000) )
    hist7, _ = np.histogram( myval2 , density=True, bins=bins, range=(0,25000))
    hist8, _ = np.histogram( myval3, density=True, bins=bins, range=(0,25000))
    X_hist.append(np.hstack((hist1,hist2,hist3,hist4,hist5,hist6,hist7,hist8)))

X_hist = np.array(X_hist)

#################### Import models #######################

from sklearn.externals import joblib #joblib is sklearn's pickle
sat_to_sensor_model = joblib.load('models/sat-to-sensor-model/sat-to-sensor-model.pkl')
sensor_to_power_mod = joblib.load('models/sensor-to-power-model/sensor-to-power-model.pkl')

X_sensor = sat_to_sensor_model.predict(X_hist)
y_power = sensor_to_power_mod.predict(X_sensor)

print y_power[0]

In [111]:
from time import sleep
sleep(1)